# 2. RAG en castelán que crea vector store a partir de un ou varios ficheiros pdf, en este caso utilizaremos llama 3.2

In [ ]:
#Instalar librerias necesarias
# !pip install langchain PyPDF2 langchain_huggingface chromadb

In [1]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)

pdf_path = "ciberseguridad.pdf"

page_content = extract_text_from_pdf(pdf_path)

chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")

documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")

/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Overwriting cache for 0 2994


Number of chunks: 84
Documents added to the vector store.


In [2]:
prompt = ChatPromptTemplate.from_template(
    template="Usa el contexto a continuación para responder la pregunta del usuario:\n\n{context}\n\nPregunta: {question}\nRespuesta:"
)

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

query = "¿Cuál es el tema principal del documento?"
response = qa_chain.invoke(query)
print("Respuesta:", response)

Respuesta: {'query': '¿Cuál es el tema principal del documento?', 'result': 'El tema principal del documento es la Gestión del Riesgo de Seguridad de la Información, específicamente cómo identificar, analizar y tratar los riesgos en este ámbito. El documento se centra en proporcionar una guía para que los empresarios puedan implementar un proceso eficaz de gestión del riesgo en su organización, desde la identificación y análisis de riesgos hasta su tratamiento y seguimiento.'}
